## Pipeline обработки запроса

### Загрузка данных и функций (достаточно выполнить 1 раз)

In [1]:
from main import setup_environment, load_data
from retriever import retriever, RetrieverOut
from planner import planner, PlannerOut
from grounder import grounder
from executor import executor

from openai import OpenAI
from config import PipelineConfig


api_key, db_path = setup_environment()
db = load_data(db_path, wave_filter=["2025-03"])    # Фильтр по последней волне

client = OpenAI(
    base_url="https://openrouter.ai/api/v1",
    api_key=api_key,
)

2025-11-12 16:46:59 | INFO     | main                 | Environment loaded: DB_PATH=C:\Users\kateu\Documents\Ivanovs Index\V2\DB\25W3\db_CORRECTED.parquet
2025-11-12 16:46:59 | INFO     | main                 | Loading data from C:\Users\kateu\Documents\Ivanovs Index\V2\DB\25W3\db_CORRECTED.parquet
2025-11-12 16:47:00 | INFO     | main                 | Full dataset shape: (26505646, 5)
2025-11-12 16:47:00 | INFO     | main                 | Filtered by waves: ['2025-03'], new shape: (672405, 5)


### Настройка используемых моделей и их температур

Если нужно поменять модель, это делается здесь

In [2]:
PPL_cfg = PipelineConfig.setup(
    df=db, client=client,
    # параметры ретривера
    retriever_params={
        "model": "openrouter/polaris-alpha",
        # "model": "alibaba/tongyi-deepresearch-30b-a3b:free",
        "temperature": 0.4
    },
    # параметры планировщика
    planner_params={
        "model": "deepseek/deepseek-chat-v3.1",
        # !!! "model": "deepseek/deepseek-chat-v3.1", # Платная модель
        "temperature": 0.4
    }
)

### Еще варианты моделей
# "alibaba/tongyi-deepresearch-30b-a3b:free"
# "meta-llama/llama-4-maverick:free"
# !!! "deepseek/deepseek-chat-v3.1" # Платная модель

### Retriever

Извлекает релевантные вопросы на основе запроса пользователя и всего набора вопросов

Пользовательский запрос

In [3]:
# user_query = "Я хочу посчитать индекс потребительской уверенности по потребителям из москвы"
# user_query = "Я хочу посчитать размер средних сбережений и норму сбережений среди тех, у кого они есть"
user_query = "Я хочу посчитать долю взаимопроникновения клиентов Чижика и Пятерочки"
# user_query = "Мне нужны все вопросы, связанные с автотранспортом (сроки владения, предпочтения по маркам, планы по покупке и пр.)"
# user_query = "Мне нужны все вопросы, связанные с автотранспортом (сроки владения, предпочтения по маркам, планы по покупке и пр.)"

Обращение к LLM

> ! Возможны проблемы парсинга ответов

In [4]:
# Сохранение, чтобы не делать 1 и тот же запрос кучу раз:
# можно 1 раз сохранить, а далее только читать

# retriever_out = retriever(user_query, PPL_cfg)
# retriever_out.save("cur_retrieved.json")

In [5]:
retriever_out = RetrieverOut.load("cur_retrieved.json")
print(retriever_out)

1. '[C6_offline] Как часто Вы совершали покупки в этих магазинах в последнем месяце? @ Чижик'
	Reason: Позволяет выделить пользователей Чижика (любая ненулевая частота покупок), что является базой для расчета доли пересечения.
2. '[C6_offline] Как часто Вы совершали покупки в этих магазинах в последнем месяце? @ Пятерочка'
	Reason: Позволяет определить, какие респонденты из базы (в том числе покупатели Чижика) являются покупателями Пятёрочки, и рассчитать долю кросс-посещения (взаимопроникновения) аудиторий.
3. '[Q115] В каких магазинах Вы делаете покупки?'
	Reason: Позволяет определить, покупает ли респондент в Пятерочке и/или Чижике вообще. На этой базе строится основная метрика взаимопроникновения аудиторий (пересечение тех, кто отметил оба магазина, к аудитории каждого из них).
4. '[S8] В каком магазине Вы покупали основную часть продуктов в последнем месяце?'
	Reason: Нужен для дополнительного анализа глубины пересечения: доля для которых Пятерочка/Чижик являются основными, но кот

### Planner

Строит план на основе пользовательского запроса и набора релевантных ответов, к которым подмешиваются их ответы

В план включаются команды из [`capability_spec.py`](./capability_spec.py)

In [7]:
# Сохранение, чтобы не делать 1 и тот же запрос кучу раз:
# можно 1 раз сохранить, а далее только читать

# planner_out = planner(user_query, retriever_out, PPL_cfg)
# planner_out.save("cur_planned.json")

In [8]:
planner_out = PlannerOut.load("cur_planned.json")
print(planner_out)

АНАЛИЗ: Для расчета доли взаимопроникновения клиентов Чижика и Пятерочки нужно определить два множества: клиенты Чижика и клиенты Пятерочки, а затем найти пересечение этих множеств. В качестве базы можно использовать вопрос о магазинах, где респонденты делают покупки ([Q115]), так как он охватывает обе сети и позволяет определить клиентов каждой из них.
0. [s1] OperationType.LOAD_DATA
	Goal: Загрузить данные за март 2025 года
	Inputs: {'waves': ['2025-03']}
	Outputs: ['dataset']
	Constraints: {}
	Depends on: []
1. [s2] OperationType.FILTER
	Goal: Отфильтровать клиентов Чижика (всех, кто указал Чижик в списке магазинов, где делает покупки)
	Inputs: {'dataset': 'dataset', 'question': '[Q115] В каких магазинах Вы делаете покупки?', 'answer_values': ['Чижик'], 'logic': 'include'}
	Outputs: ['chizhik_customers']
	Constraints: {'question': 'MUST be from allowed_questions', 'answer_values': 'MUST be from allowed answers for the question'}
	Depends on: ['s1']
2. [s3] OperationType.FILTER
	Goal

### Grounder

Привязка шагов плана к имеющимся функциям (из [`operations.py`](./operations.py))

In [9]:
grounder_out = grounder(planner_out)

2025-11-12 16:47:26 | INFO     | grounder             | Starting grounder
2025-11-12 16:47:26 | DEBUG    | grounder             | Grounding step 1/6: s1
2025-11-12 16:47:26 | DEBUG    | grounder             | Successfully grounded step s1: LOAD_DATA
2025-11-12 16:47:26 | DEBUG    | grounder             | Grounding step 2/6: s2
2025-11-12 16:47:26 | DEBUG    | grounder             | Successfully grounded step s2: FILTER
2025-11-12 16:47:26 | DEBUG    | grounder             | Grounding step 3/6: s3
2025-11-12 16:47:26 | DEBUG    | grounder             | Successfully grounded step s3: FILTER
2025-11-12 16:47:26 | DEBUG    | grounder             | Grounding step 4/6: s4
2025-11-12 16:47:26 | DEBUG    | grounder             | Successfully grounded step s4: FILTER
2025-11-12 16:47:26 | DEBUG    | grounder             | Grounding step 5/6: s5
2025-11-12 16:47:26 | DEBUG    | grounder             | Successfully grounded step s5: PIVOT
2025-11-12 16:47:26 | DEBUG    | grounder             | Gro

### Executor

Валидирует план, осуществляет топологическую сортировку, выполняет последовательность шагов

In [10]:
ctx = {"dataset": db}
ctx = executor(grounder_out, ctx)

2025-11-12 16:47:33 | INFO     | executor             | Starting executor with 6 steps
2025-11-12 16:47:33 | DEBUG    | executor             | Validating step dependencies
2025-11-12 16:47:33 | DEBUG    | executor             | Performing topological sort
2025-11-12 16:47:33 | DEBUG    | executor             | Topological sort completed: ['s1', 's2', 's3', 's4', 's5', 's6']
2025-11-12 16:47:33 | DEBUG    | executor             | Dependencies validation passed
2025-11-12 16:47:33 | DEBUG    | executor             | Performing topological sort
2025-11-12 16:47:33 | DEBUG    | executor             | Topological sort completed: ['s1', 's2', 's3', 's4', 's5', 's6']
2025-11-12 16:47:33 | INFO     | executor             | Steps execution order: ['s1', 's2', 's3', 's4', 's5', 's6']
2025-11-12 16:47:33 | INFO     | executor             | Executing step 1/6: s1 (LOAD_DATA)
2025-11-12 16:47:33 | DEBUG    | executor             |   waves = ['2025-03'] (literal)
2025-11-12 16:47:33 | DEBUG    | exe

Далее можно извлекать созданные таблицы из `ctx`

In [11]:
opts = [f"{i}. '{n}'" for i, n in enumerate(ctx.keys(), start=1)]
opts = "\n".join(opts)

print(f"Доступные варианты:\n{opts}")

Доступные варианты:
1. 'dataset'
2. 'filtered_dataset'
3. 'chizhik_customers'
4. 'pyaterochka_customers'
5. 'intersection_customers'
6. 'pivot'
7. 'chizhik_pivot'
8. 'intersection_pivot'


Пока что извлекать и смотреть можно только ручками...

In [ ]:
ctx["intersection_pivot"]